In [1]:
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.spark:spark-sql-kafka-0-10_2.12::3.1.3 pyspark-shell"
import findspark
findspark.init()
import pyspark
findspark.find()

'E:\\spark-3.0.2-bin-hadoop2.7'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,TimestampType, DoubleType, StringType, StructField
from dotenv import load_dotenv
import os
load_dotenv()

mongo_db = os.getenv("mongoDB")
KAFKA_TOPIC_NAME_CONS = "twitter_streaming"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

In [4]:
spark = SparkSession.\
        builder\
        .master("local[*]")\
        .appName("TwitterStreaming")\
        .getOrCreate()
spark

In [5]:
 spark.sparkContext.setLogLevel("ERROR")

In [6]:
schema = StructType() \
        .add("data", StructType() \
            .add("created_at", StringType())
            .add("id", StringType())
            .add("text", StringType()))

In [7]:
# Construct a streaming DataFrame that reads from testtopic
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
  .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
  .load() \
  .select(F.col('key').cast('string'),
            F.from_json(F.col("value").cast("string"), 
                schema)['data']['created_at'].alias('created_at'),
            F.from_json(F.col("value").cast("string"), 
                schema)['data']['text'].alias('text')) 

In [8]:
df.printSchema()

root
 |-- key: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)



In [8]:
query= df.writeStream \
    .format("console") \
            .outputMode("append") \
            .start() \
            .awaitTermination()
display(df)